In [1]:
import epidag as dag
import logging
import sys

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
scr = '''
PCore test {
    al = 1
    be = 1
    prob ~ beta(al, be)
    x ~ binom(n, prob)    
}
'''

bn = dag.bayes_net_from_script(scr)

In [3]:
data = [
    {'id': 1, 'n': 10, 'x': 4},
    {'id': 2, 'n': 20, 'x': 7}
]

dm = dag.as_data_model(bn, data)

In [4]:
p = dm.sample_prior()
dm.evaluate_likelihood(p)

-7.231722288888591

In [5]:
fit = dag.fitting.SIR(dm)
fit.fit(1000)
fit.report_posterior()

INFO:epidag.fitting.alg.sir:Initialising
INFO:epidag.fitting.alg.sir:Sampling-Importance
INFO:epidag.fitting.alg.sir:Resampling


,al,be,prob
count,1000.0,1000.0,1000.000000
mean,1.0,1.0,0.367267
std,0.0,0.0,0.082388
min,1.0,1.0,0.180282
25%,1.0,1.0,0.306562
50%,1.0,1.0,0.360147
75%,1.0,1.0,0.411805
max,1.0,1.0,0.650308


In [6]:
fit = dag.fitting.MCMC(dm)
fit.fit(1000)
fit.report_posterior()

INFO:epidag.fitting.alg.mcmc:Initialising
INFO:epidag.fitting.alg.mcmc:Burning in
INFO:epidag.fitting.alg.mcmc:Gathering posteriori
INFO:epidag.fitting.alg.mcmc:Completed


,al,be,prob
count,1000.0,1000.0,1000.000000
mean,1.0,1.0,0.367674
std,0.0,0.0,0.083025
min,1.0,1.0,0.149345
25%,1.0,1.0,0.303206
50%,1.0,1.0,0.370402
75%,1.0,1.0,0.423577
max,1.0,1.0,0.639373


In [7]:
fit = dag.fitting.ABC(dm)
fit.fit(100)
fit.report_posterior()

INFO:epidag.fitting.alg.abc:Initialising
INFO:epidag.fitting.alg.abc:Testing threshold
INFO:epidag.fitting.alg.abc:Fitting
INFO:epidag.fitting.alg.abc:Gathering posteriori


,al,be,prob
count,100.0,100.0,100.000000
mean,1.0,1.0,0.366891
std,0.0,0.0,0.014608
min,1.0,1.0,0.342244
25%,1.0,1.0,0.354346
50%,1.0,1.0,0.366568
75%,1.0,1.0,0.378838
max,1.0,1.0,0.390891


In [9]:
scr = '''
PCore ex {
    sun ~ unif(0, 1)
    b0 = 1
    heat ~ exp(sun)
    x1 ~ norm(0, 1)
    x2 ~ norm(0, 1)
    b1 ~ unif(10, 20)
    mu1 = b0 + b1*x1 + b2*x2
    mu2 = b0 + b1*x1
    y1 ~ norm(mu1, sd)
    y2 ~ norm(mu2, 1)
}

'''

bn = dag.bayes_net_from_script(scr)
hie = {
    'g': ['sun', 'agent'],
    'agent': ['x1', 'x2','mu1', 'mu2']
}
sm = dag.as_simulation_core(bn, hie=hie, random=['x2'])


h = dag.form_hierarchy(bn, hie)

h.print()

g(heat, b2, sd, b1, b0, sun)
-- agent(mu1, y2, x1, mu2, y1, x2)


In [10]:
pc = sm.generate(exo={'sd'})
ag1 = pc.breed('a1', 'agent')

In [19]:
ag1.get_sampler('y1')(b2= 4, sd =3)

-1.5826879957652007

In [13]:
pc.get_sampler('heat')()

0.7308904995335411

In [12]:
ng = dag.form_hierarchy(bn, hie)
dag.analyse_node_type(bn, ng, True)


Group g
Exogenous    : ['b2', 'sd']
Can be fixed: ['b1', 'b0', 'sun', 'heat']
Can be random: ['sun', 'heat']
Can be actors: ['heat']
--Group agent
  Exogenous    : []
  Can be fixed: ['x1', 'mu2', 'x2', 'y2']
  Can be random: ['x1', 'mu2', 'x2', 'y2', 'y1', 'mu1']
  Can be actors: ['y2', 'y1']


{'g': (['b2', 'sd'], ['b1', 'b0'], ['sun'], ['heat'], [], []),
 'agent': ([], [], ['x1', 'mu2', 'x2'], ['y2'], ['y1'], ['mu1'])}